# Feature Extraction for Metal Nanoparticle Surface Atoms Clustering

This jupyter notebook is used to extract various features from metal nanoparticle data sets. Note that the data sets here are individual nanoparticles, and instances are individual atoms. The features are extracted using NCPac, SYMMOL, and Python functions (with aid from packages such as PySCal).

## Index:
- [Requirements](#requirements)
- [Import Libraries](#import-libraries)
- [Feature Extraction](#feature-extract)
    - [NCPac Output](#ncpac-output)
    - [PySCal Output](#pyscal)
    - [Point Group Symmetry](#pgsym)
    - [Element Specific](#ele-spec)
    - [Combine Features](#combine-feat)
- [Variables Storage](#var-store)

<a id='requirements'></a>
## Requirements

The following files should be available in a specified directory (here it is ./data/NCPacFeats/):
* "{X}.xyz"
* "{X}_od_FEATURESET.csv"
* "{X}_ov_BOND_length.xyz"
* "{X}_ov_BOND_angle1.xyz"
* "{X}_ov_BOND_angle2.xyz"
* "{X}_ov_BOND_torsion.xyz"
* "{X}_ov_COORD_total.xyz"
* "{X}_ov_RADIAL_distance.xyz"
* "{X}_ov_Q6Q6.xyz"
* "{X}_ov_COORD_surface.xyz"
* "{X}_ov_SURF_layer.xyz"

where X = abbreviation of the target metallic nanoparticle

<a id='import-libraries'></a>
## Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# %%capture
import math
import os
import pickle
import random
import re
import sys

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import rcParams
import numpy as np
import pandas as pd
import pyscal.core as pc
# import modin.pandas as pd
import seaborn as sns

In [3]:
# Set up notebook environment
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [4]:
# Print version for reproducibility
print(f"numpy version {np.__version__}")
print(f"pandas version {pd.__version__}")
print(f"seaborn version {sns.__version__}")
np.random.seed(777)

numpy version 1.21.5
pandas version 1.3.5
seaborn version 0.11.2


<a id='feature0-extraction'></a>
## Feature Extraction

In [5]:
featDirPath = "./data/NCPacFeats"  # To be modified accordingly if needed
SYMMOLDirPath = "./symmol"  # To be modified accordingly if needed
testCases = ["COS1T000", "ICS1T000", "TOS1T000", 
             "CUS1T000", "CUS2T000", "CUS3T000", 
             "DHS1T000", "DHS2T000", "DHS3T000",  # DHS1T000 flagged, abnormal bond length statistics -- average bond length longer than others!!
             "OTS1T000", "OTS2T000", "OTS3T000", 
             "RDS1T000", "RDS2T000", "RDS3T000", 
             "THS1T000", "THS2T000", "THS3T000", 
             "OTS1T323", "OTS2T323", "OTS3T323", 
             "RDS1T323", "RDS2T323", "RDS3T323", 
             "THS1T323", "THS2T323", "THS3T323", 
             "OTS1T523", "OTS2T523", "OTS3T523", 
             "RDS1T523", "RDS2T523", "RDS3T523", 
             "THS1T523", "THS2T523", "THS3T523", 
             "disPd1", "disPd2", "disPd3"]
%store testCases

Stored 'testCases' (list)


In [6]:
# all_directories = list(map(lambda x: os.path.join(base_path, x.split(':')[0], x.split(':')[1]), list(df_org.index)))
featSetCases = []
for (i, testCase) in enumerate(testCases):
    print(i, testCase)
    featSetCases.append(pd.read_csv(f"{featDirPath}/{testCases[i]}_od_FEATURESET.csv", header=1))
featSetCases[-1].head()

0 COS1T000
1 ICS1T000
2 TOS1T000
3 CUS1T000
4 CUS2T000
5 CUS3T000
6 DHS1T000
7 DHS2T000
8 DHS3T000
9 OTS1T000
10 OTS2T000
11 OTS3T000
12 RDS1T000
13 RDS2T000
14 RDS3T000
15 THS1T000
16 THS2T000
17 THS3T000
18 OTS1T323
19 OTS2T323
20 OTS3T323
21 RDS1T323
22 RDS2T323
23 RDS3T323
24 THS1T323
25 THS2T323
26 THS3T323
27 OTS1T523
28 OTS2T523
29 OTS3T523
30 RDS1T523
31 RDS2T523
32 RDS3T523
33 THS1T523
34 THS2T523
35 THS3T523
36 disPd1
37 disPd2
38 disPd3


,Frame,N Total,N Bulk,N Surf,Rmin,Rmax,Rdif,Ravg,Rstd,Rskew,Rkurt,100,111,110,311,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100.1,101,102,103,104,105,106,107,108,109,110.1,111.1,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,1.1,2.1,3.1,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,Type,Avg Total,Avg Bulk,Avg Surf,Avg Surfo,Avg GTotal,Avg GBulk,Avg GSurf,Avg GSurfo,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,B0,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S20,S0o,S1o,S2o,S3o,S4o,S5o,S6o,S7o,S8o,S9o,S10o,S11o,S12o,S13o,S14o,S15o,S16o,S17o,S18o,S19o,S20o,GT0,GT1,GT2,GT3,GT4,GT5,GT6,GT7,GT8,GT9,GT10,GT11,GT12,GT13,GT14,GT15,GT16,GT17,GT18,GT19,GT20,GB0,GB1,GB2,GB3,GB4,GB5,GB6,GB7,GB8,GB9,GB10,GB11,GB12,GB13,GB14,GB15,GB16,GB17,GB18,GB19,GB20,GS0,GS1,GS2,GS3,GS4,GS5,GS6,GS7,GS8,GS9,GS10,GS11,GS12,GS13,GS14,GS15,GS16,GS17,GS18,GS19,GS20,GS0o,GS1o,GS2o,GS3o,GS4o,GS5o,GS6o,GS7o,GS8o,GS9o,GS10o,GS11o,GS12o,GS13o,GS14o,GS15o,GS16o,GS17o,GS18o,GS19o,GS20o,Type.1,Type.2,Avg Length,Std Dev,Max Length,Min Length,N Lengths,Type.3,Type.4,Avg Length.1,Std Dev.1,Max Length.1,Min Length.1,N Lengths.1,1 1,N Bonds,Type.5,Type.6,0.0000,0.0800,0.1600,0.2400,0.3200,0.4000,0.4800,0.5600,0.6400,0.7200,0.8000,0.8800,0.9600,1.0400,1.1200,1.2000,1.2800,1.3600,1.4400,1.5200,1.6000,1.6800,1.7600,1.8400,1.9200,2.0000,2.0800,2.1600,2.2400,2.3200,2.4000,2.4800,2.5600,2.6400,2.7200,2.8000,2.8800,2.9600,3.0400,3.1200,3.2000,3.2800,3.3600,3.4400,3.5200,3.6000,3.6800,3.7600,3.8400,3.9200,4.0000,4.0800,4.1600,4.2400,4.3200,4.4000,4.4800,4.5600,4.6400,4.7200,4.8000,4.8800,4.9600,5.0400,5.1200,5.2000,5.2800,5.3600,5.4400,5.5200,5.6000,5.6800,5.7600,5.8400,5.9200,6.0000,6.0800,6.1600,6.2400,6.3200,6.4000,6.4800,6.5600,6.6400,6.7200,6.8000,6.8800,6.9600,7.0400,7.1200,7.2000,7.2800,7.3600,7.4400,7.5200,7.6000,7.6800,7.7600,7.8400,7.9200,8.0000,8.0800,8.1600,8.2400,8.3200,8.4000,8.4800,8.5600,8.6400,8.7200,8.8000,8.8800,8.9600,9.0400,9.1200,9.2000,9.2800,9.3600,9.4400,9.5200,9.6000,9.6800,9.7600,9.8400,9.9200,10.0000,10.0800,10.1600,10.2400,10.3200,10.4000,10.4800,10.5600,10.6400,10.7200,10.8000,10.8800,10.9600,11.0400,11.1200,11.2000,11.2800,11.3600,11.4400,11.5200,11.6000,11.6800,11.7600,11.8400,11.9200,0.0000.1,0.0767,0.1534,0.2301,0.3068,0.3835,0.4602,0.5369,0.6136,0.6903,0.7670,0.8437,0.9204,0.9971,1.0738,1.1505,1.2272,1.3039,1.3806,1.4573,1.5340,1.6107,1.6874,1.7641,1.8408,1.9175,1.9942,2.0709,2.1476,2.2243,2.3010,2.3777,2.4544,2.5311,2.6078,2.6845,2.7612,2.8379,2.9146,2.9913,3.0680,3.1447,3.2214,3.2981,3.3748,3.4515,3.5282,3.6049,3.6816,3.7583,3.8350,3.9117,3.9884,4.0650,4.1417,4.2184,4.2951,4.3718,4.4485,4.5252,4.6019,4.6786,4.7553,4.8320,4.9087,4.9854,5.0621,5.1388,5.2155,5.2922,5.3689,5.4456,5.5223,5.5990,5.6757,5.7524,5.8291,5.9058,5.9825,6.0592,6.1359,6.2126,6.2893,6.3660,6.4427,6.5194,6.5961,6.6728,6.7495,6.8262,6.9029,6.9796,7.0563,7.1330,7.2097,7.2864,7.3631,7.4398,7.5165,7.5932,7.6699,7.7466,7.8233,7.9000,7.9767,8.0534,8.1301,8.2068,8.2835,8.3602,8.4369,8.5136,8.5903,8.6670,8.7437,8.8204,8.8971,8.9738,9.0505,9.1272,9.2039,9.2806,9.3573,9.4340,9.5107,9.5874,9.6641,9.7408,9.8175,9.8942,9.9709,10.0476,10.1243,10.2010,10.2777,10.3544,10.4311,10.5078,10.5845,10.6612,10.7379,10.8146,10.8913,10.9680,11.0447,11.1214,11.1981,11.2748,11.3515,11.4282,11.5049,11.5816,11.6583,11.7350,11.8117,11.8884,11.9651,12.0417,12.1184,12.1951,12.2718,12.348

<a id='ncpac-output'></a>
### NCPac Output

In [7]:
def NCPacFeat(testCase):
    """
    Extract features computed by NCPac, including:
        - element type
        - spatial coordinates (x, y, z)
        - bond length statistics (avg, max, min, num)
        - 1st order bond angle statistics (avg, max, min, num)
        - 2nd order bond angle statistics (avg, max, min, num)
        - bond torsion statistics (avg, max, min, num for pos, neg angles)
        - (generalised) coordination number
        - radial distance from nanoparticle centre
        - q6q6 value
        - surface atom (generalised) coordination number
        - surface layer membership (Boolean)
    Inputs: 
        testCase = name of test case nanoparticle
    Outputs: 
        beList, coordList, blList, baList, btList, CNList, radList, q6q6List, surfCNList, surfList =  DataFrames or Series of relevant feature values of each atom
    """
    # Extract the values from output files generated by NCPac
    beList = pd.read_csv(f"{featDirPath}/{testCase}_ov_BOND_length.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, 0]
    coordList = pd.read_csv(f"{featDirPath}/{testCase}_ov_BOND_length.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, 1:4]
    blList = pd.read_csv(f"{featDirPath}/{testCase}_ov_BOND_length.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, 4:]
    ba1List = pd.read_csv(f"{featDirPath}/{testCase}_ov_BOND_angle1.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, 4:]
    ba2List = pd.read_csv(f"{featDirPath}/{testCase}_ov_BOND_angle2.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, 4:]
    btList = pd.read_csv(f"{featDirPath}/{testCase}_ov_BOND_torsion.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, 4:]
    CNList = pd.read_csv(f"{featDirPath}/{testCase}_ov_COORD_total.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, -2:]
    radList = pd.read_csv(f"{featDirPath}/{testCase}_ov_RADIAL_distance.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, -1]
    q6q6List = pd.read_csv(f"{featDirPath}/{testCase}_ov_Q6Q6.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, -1]
    surfCNList = pd.read_csv(f"{featDirPath}/{testCase}_ov_COORD_surface.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, -2:]
    surfList = pd.read_csv(f"{featDirPath}/{testCase}_ov_SURF_layer.xyz", sep='\s+|\t+|\s+\t+|\t+\s+', header=None, skiprows=2, engine='python').iloc[:, -1]
    
    # Rename the columns
    coordList.columns = ['x', 'y', 'z']
    blList.columns = ['blavg', 'blmax', 'blmin', 'blnum']
    ba1List.columns = ['ba1avg', 'ba1max', 'ba1min', 'ba1num']
    ba2List.columns = ['ba2avg', 'ba2max', 'ba2min', 'ba2num']
    btList.columns = ['btposavg', 'btnegavg', 'btnegmax', 'btnegmin', 'btposmax', 'btposmin', 'btposnum', 'btnegnum']
    beList.rename('ele', axis=None, copy=False, inplace=True, level=None, errors='ignore')
    CNList.columns = ['cn', 'gcn']
    radList.rename('rad', axis=None, copy=False, inplace=True, level=None, errors='ignore')
    q6q6List.rename('q6q6', axis=None, copy=False, inplace=True, level=None, errors='ignore')
    surfCNList.columns = ['scn', 'sgcn']
    surfList.rename('surf', axis=None, copy=False, inplace=True, level=None, errors='ignore')
    
    return beList, coordList, blList, ba1List, ba2List, btList, CNList, radList, q6q6List, surfCNList, surfList

<a id='pyscal'></a>
### PySCal Output

In [8]:
def runPYSCAL(system, testCase, latConst=4.065, multEle=False):
    """
    Extract features computed by PySCal, including:
        - Steinhardt's parameters
        - disorder parameters (separate function)
        - angular parameters
        - centrosymmetry parameters (separate function)
        - entropy-enthalpy parameters
        - Cowley short range parameters
    Inputs: 
        testCase = name of test case nanoparticle
        latConst = lattice constant, default for gold
        multEle = multi-elemental indicator (Boolean)
    Outputs: 
        system = PySCal object containing the whole system
    """
    system.calculate_q(q=[2, 4, 6, 8, 10, 12], averaged=True, only_averaged=False, condition=None, clear_condition=False)
    system.calculate_angularcriteria()
    system.calculate_chiparams(angles=False)
    system.calculate_entropy(rm=1.4*latConst, sigma=0.2, rstart=0.001, h=0.001, local=False, M=12, N=6, ra=None, averaged=True, 
                             switching_function=False)
    if multEle: 
        system.calculate_sro(reference_type=1, average=True, shells=2)
    return system

In [9]:
def pyscalFeat(testCase, CNList):
    """
    Run PySCal to extract features useful for structural distinction
    Inputs: 
        testCase = name of test case nanoparticle
    Outputs: 
        pyscalList = DataFrames of all feature values of each atom
        system = PySCal object containing system of atoms
    """
    method, cutoff = 'cutoff', 'adaptive'  # SANN and Voronoi (especially worse) neighbour finding schemes fail for surface atoms
    system = pc.System()
    system.read_inputfile(f"{featDirPath}/{testCase}.dump")
    system.find_neighbors(method=method, cutoff=cutoff, threshold=2, filter=None, padding=1.2, nlimit=6)
    system = runPYSCAL(system=system, testCase=testCase, latConst=3.89, multEle=False)

    # Extract angular, chi, entropy-enthalpy parameters
    angularList, entropyList, entropyAvgList, chiList = [], [], [], []
    for (i, atom) in enumerate(system.atoms):
        angularList.append(atom.angular)
        entropyList.append(atom.entropy)
        entropyAvgList.append(atom.avg_entropy)
        chiList.append(atom.chiparams)
    angEntList = pd.DataFrame({"angParam": angularList, "entroParam": entropyList, "entroAvgParam": entropyAvgList})
    chiList = pd.DataFrame(chiList, columns = ["chi1", "chi2", "chi3", "chi4", "chi5", "chi6", "chi7", "chi8", "chi9"])
        
    # Extract Steinhardt's and disorder parameters for each neighbour definition
    qVals = [2, 4, 6, 8, 10, 12]
    q = system.get_qvals(qVals, averaged=False)
    qAvg = system.get_qvals(qVals, averaged=True)
    qList = pd.DataFrame.from_records(list(map(list, zip(*q))), 
                                      columns=["q2", "q4", "q6", "q8", "q10", "q12"])
    qAvgList = pd.DataFrame.from_records(list(map(list, zip(*qAvg))), 
                                      columns=["q2avg", "q4avg", "q6avg", "q8avg", "q10avg", "q12avg"])
    pyscalDFs = [angEntList, chiList, qList, qAvgList]
    for (i, q) in enumerate(qVals):
        system.calculate_disorder(averaged=True, q=q)
        disordList = pd.DataFrame([(atom.disorder, atom.avg_disorder) for atom in system.atoms], columns=[f"disord{q}", f"disordAvg{q}"])
        pyscalDFs.append(disordList)
        
    # Compute centrosymmetry parameter for each atom according to its coordination (has to be last since it recalculates neighbour list)
    uniqueEvenCNs = []
    for cn in CNList['cn'].unique(): 
        uniqueEvenCNs.append(cn) if cn % 2 == 0 else uniqueEvenCNs.append(cn + 1)
    centArr = np.zeros(len(CNList))
    for cn in set(uniqueEvenCNs):
        system.calculate_centrosymmetry(nmax=cn, get_vals=True)
        for idx in CNList[CNList['cn'].isin([cn, cn-1])].index:
            centArr[idx] = system.atoms[idx].centrosymmetry
    centList = pd.DataFrame(centArr, columns=['centParam'])
    pyscalDFs.append(centList)

    pyscalList = pd.concat(objs=pyscalDFs, axis=1, ignore_index=False, join='inner')
    return pyscalList, system

#### PySCal
This is a python module for the calculation of local atomic structural environments including Steinhardt’s bond orientational order parameters during post-processing of atomistic simulation data. Steinhardt’s order parameters are widely used for identification of crystal structures. They are also used to identify if an atom is solid or liquid. Functionality of PySCal:
- calculation of Steinhardt’s order parameters and their averaged version and disorder parameters.
- calculation of Steinhardt parameters weighted using face area of Voronoi polyhedra.
- classification of atoms as solid or liquid.
- clustering of particles based on a user defined property.
- methods for calculating radial distribution function, voronoi volume of particles, number of vertices and face area of voronoi polyhedra and coordination number.
- calculation of angular parameters such as for identification of diamond structure and Ackland-Jones angular parameters.
- calculation of centrosymmetry parameter for identification of defects.
- calculation of adaptive common neighbor analysis for identification of crystal structures.

<a id='pgsym'></a>
### Point Group Symmetry

In [10]:
def runSYMMOL(inpFile, nList, atomIdx, dcm, dcme):
    """
    Run SYMMOL to identify matching point group symmetry of targeted atom (taking 1st nearest neighbours into account)
    Inputs: 
        inpFile = xyz-formatted file of test case nanoparticle
        nList = list of neighbouring atoms of targetted atom
        atomIdx = index of targeted atom
        dcm = tolerance to search for point group symmetry
        dcme = tolerance to search for point group symmetry after taking IMMOL=-1 into account (Not varied for now)
    Outputs: 
        Boolean = whether any matching point group symmetry has been found within tolerance
    """
    # Generate SYMMOL input file and record the current metal cluster in xyz format
    with open(f"{featDirPath}/{inpFile}", "r") as f1:
        with open(f"{SYMMOLDirPath}/neighClust.xyz", "w") as f2:
            with open(f"{SYMMOLDirPath}/symmol.inp", "w") as f3:
                line1 = f1.readline()
                line2 = f1.readline()
                f2.write(line1)
                f2.write(line2)
                f3.write(f"1 1 1 90 90 90\n2 1 {dcm} {dcme}\n")
                for (i, line) in enumerate(f1.readlines()): 
                    if i == atomIdx or i in nList:
                        f2.write(line.strip() + " 1\n")
                        lineEle = line.split()
                        # Can include element type below
                        symmolLine = f"At     1  {float(lineEle[1]):9.3f}  {float(lineEle[2]):9.3f}  {float(lineEle[3]):9.3f}\n"
                        f3.write(symmolLine)
                    else:
                        f2.write(line.strip() + " 0\n")
    
    # Run SYMMOL
    !cd ./symmol; ./symmol.exe  # To be modified depending on user's machine
    
    # Check if any point group symmetry has been identified
    with open(f"{SYMMOLDirPath}/symmol.out", "r") as f:
        for line in f.readlines():
            if line.startswith(" NO SYMMETRY EXISTS"): 
                return False
    
    return True

In [11]:
def symAtomFeat(inpFile, nList, atomIdx):
    """
    Extract features related to point group symmetries of targeted atom
    Inputs:
        inpFile = xyz-formatted file of targeted nanoparticle
        nList = list of neighbouring atoms of targetted atom
        atomIdx = index of targeted atom
    Outputs: 
        symFeatDict = dictionary containing relevant feature values of targeted atom
    """
    dcm, dcme, foundSym = 0.0, 0.1, False
    while not foundSym:
        dcm += 0.1
        foundSym = runSYMMOL(inpFile, nList, atomIdx, dcm, dcme)
        if dcm >= 10.0: 
            break
        # print("atom: {0} DCM: {1:.1f}".format(atomIdx, dcm))  # For debugging
    symFeatDict = {}
    hasTypeLines = False
    if foundSym:
        with open(f"{SYMMOLDirPath}/symmol.out", "r") as f:
            lines = f.readlines()
            for (i, line) in enumerate(lines):
                if line.startswith(" PRINCIPAL INERTIA MOMENTS"):
                    inerMomLine = lines[i+1].split()
                    symFeatDict["Ixx"] = symFeatDict.get("Ixx", 0) + float(inerMomLine[0])
                    symFeatDict["Iyy"] = symFeatDict.get("Iyy", 0) + float(inerMomLine[1])
                    symFeatDict["Izz"] = symFeatDict.get("Izz", 0) + float(inerMomLine[2])
                    symFeatDict["degenDeg"] = symFeatDict.get("degenDeg", 0) + int(inerMomLine[-1])
                elif line.startswith(" Schoenflies"):
                    symFeatLine = line.split()
                    symFeatDict["pg"] = symFeatDict.get("pg", "") + symFeatLine[3]
                    symFeatDict["csm"] = symFeatDict.get("csm", 0) + float(symFeatLine[6])
                    symFeatDict["molRMS"] = symFeatDict.get("molRMS", 0) + float(symFeatLine[10])
                elif "TYPE" in line:
                    hasTypeLines = True
                    symEle = line.split("TYPE ")[-1].strip()
                    symFeatDict[symEle] = symFeatDict.get(symEle, 0) + 1
                elif not hasTypeLines and "[" in line and "]" in line:
                    symEleList = re.findall('\[(...)\]', line)
                    for symEleSpace in symEleList:
                        symEle = symEleSpace.strip()
                        symFeatDict[symEle] = symFeatDict.get(symEle, 0) + 1
    return symFeatDict

In [12]:
def symFeat(testCase, system):
    """
    Extract features related to point group symmetries of each atom.
    ** Yet to check why the function misses last few atoms for some nanoparticles!!**
    Inputs: 
        testCase = name of test case nanoparticle
        system = pyscal object containing system of atoms
    Outputs: 
        symList = pandas DataFrame containing relevant feature values of each atom
    """
    # Generate feature dictionary
    method, cutoff = 'cutoff', 'adaptive'  # SANN and Voronoi (especially worse) neighbour finding schemes fail for surface atoms
    system.find_neighbors(method=method, cutoff=cutoff, threshold=2, filter=None, padding=1.2, nlimit=6)  # Remove influence from calculate_centrosymmetry
    progMark = int(len(system.atoms) / 10)
    for idx in range(len(system.atoms)):
        if idx % progMark == 0: print(f"      Progress: {idx / progMark * 10} %")
        nList = system.atoms[idx].neighbors
        symFeatDict = symAtomFeat(inpFile=f"{testCase}.xyz", nList=nList, atomIdx=idx)
        symFeatDF = pd.DataFrame.from_dict(data=symFeatDict, orient='index', dtype=None)
        symList = symFeatDF.transpose() if idx == 0 else pd.concat(objs=[symList, symFeatDF.transpose()], axis=0, ignore_index=True, join='outer')
        
    # Fill the missing values for symmetry operations with zeros
    symEleCols = symList.columns[7:]
    for col in symEleCols: symList[col].fillna(value=0, method=None, axis=0, inplace=True, limit=None, downcast=None)
        
    # Add relevant columns to point groups with infinite number of planes or infinite symmetry
    symOpsDicts = {"C(inf)v": {"symOps": ["C(inf)", "Cs"], "numOps": (2, np.inf)}, 
                  "D(inf)h": {"symOps": ["C(inf)", "Cs", "Ci", "S(inf)", "C2"], "numOps": (2, np.inf, 1, 2, np.inf)}}
    for pgSym in symOpsDicts.keys ():
        if pgSym in symList["pg"].unique(): 
            symOps, numOps = symOpsDicts[pgSym]["symOps"], symOpsDicts[pgSym]["numOps"]
            for symOp in symOps:
                if symOp not in symList: symList[symOp] = 0
            symList.loc[symList["pg"] == pgSym, symOps] = numOps
    
    return symList

<a id='ele-spec'></a>
### Element Specific

In [13]:
def eleFeat(beList):
    """
    Extract features related to element type of each atom
    Inputs: 
        beList = pandas Series of the element type of each atom
    Outputs: 
        eleList = pandas DataFrame containing relevant feature values of each atom
    eleFeatCols = ["amas", "anum", "aden", 
                   "vdwrad", "metrad", "covrad", 
                   "tote", "totp", "totn", 
                   "1s", "2s", "2p", "3s", "3p", "3d", "4s", "4p", "4d", "4f", "5s", "5p", "5d", "5f", "6s", 
                   "ionE1", "ionE2", "eleneg", "stanpot", 
                   "mp", "bp"]
    units = ["g/mol", "-", "g/cm^3 at 293 K", 
            "A", "A", "A", "A", 
            "-", "-", "-", "-", 
            "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", 
            "kJ/mol", "kJ/mol", "kJ/mol", "-", "V", 
            "K", "K"]
    Sources: https://www.lenntech.com/periodic/elements/pt.htm
    """
    eleAtomFeatDict = {"Au": {"amas": 196.9655, "anum": 79, "aden": 19.3, 
                              "vdwrad": 1.66, "metrad": 1.44, "covrad": 1.44, 
                              "tote": 79, "totp": 79, "totn": 118, 
                              "1s": 2, "2s": 2, "2p": 6, "3s": 2, "3p": 6, "3d": 10, "4s": 2, "4p": 6, "4d": 10, "4f": 14, "5s": 2, "5p": 6, "5d": 10, "5f": 0, "6s": 1, 
                              "ionE1": 888, "ionE2": 1974.6, "eleneg": 2.54, "stanpot": 1.68,
                              "mp": 1337, "bp": 2273},
                       "Co": {"amas": 58.9332, "anum": 27, "aden": 8.9, 
                              "vdwrad": 2.00, "metrad": 1.25, "covrad": 1.26, 
                              "tote": 27, "totp": 27, "totn": 32, 
                              "1s": 2, "2s": 2, "2p": 6, "3s": 2, "3p": 6, "3d": 7, "4s": 2, "4p": 0, "4d": 0, "4f": 0, "5s": 0, "5p": 0, "5d": 0, "5f": 0, "6s": 0, 
                              "ionE1": 757 , "ionE2": 1666.3, "eleneg": 1.88, "stanpot": -0.28,
                              "mp": 1768, "bp": 3200},
                       "Pd": {"amas": 106.42, "anum": 46, "aden": 11.9, 
                              "vdwrad": 1.63, "metrad": 1.37, "covrad": 1.31, 
                              "tote": 46, "totp": 46, "totn": 60, 
                              "1s": 2, "2s": 2, "2p": 6, "3s": 2, "3p": 6, "3d": 10, "4s": 2, "4p": 6, "4d": 10, "4f": 0, "5s": 0, "5p": 0, "5d": 0, "5f": 0, "6s": 0, 
                              "ionE1": 703, "ionE2": 1870, "eleneg": 2.2, "stanpot": 0.85, 
                              "mp": 1768, "bp": 3200},
                       "Pt": {"amas": 195.09, "anum": 78, "aden": 21.4, 
                              "vdwrad": 1.75, "metrad": 1.385, "covrad": 1.28, 
                              "tote": 78, "totp": 78, "totn": 117, 
                              "1s": 2, "2s": 2, "2p": 6, "3s": 2, "3p": 6, "3d": 10, "4s": 2, "4p": 6, "4d": 10, "4f": 14, "5s": 2, "5p": 6, "5d": 9, "5f": 0, "6s": 1, 
                              "ionE1": 867, "ionE2": 1788, "eleneg": 1.88, "stanpot": 1.2, 
                              "mp": 1768, "bp": 3200}}
    eleFeatDFs = []
    for (i, ele) in enumerate(beList['ele']):
        if ele not in eleAtomFeatDict.keys():
            print("    Element not among (Au, Co, Pd, Pt)! Default to Au")
            ele = "Au"
        eleFeatDict = eleAtomFeatDict[ele]
        eleFeatDF = pd.DataFrame.from_dict(data=eleFeatDict, orient="index", dtype=None)
        eleFeatDFs.append(eleFeatDF.transpose().infer_objects())
    eleList = pd.concat(objs=eleFeatDFs, axis=0, ignore_index=True, join="outer")
    return eleList

<a id='combine-feat'></a>
### Combine Features

In [14]:
def allFeat(testCase):
    """
    Extract features related to element type of each atom
    Inputs: 
        testCase = name of test case nanoparticle
    Outputs: 
        dfAll = pandas DataFrame containing all relevant feature values of each atom in the test case nanoparticle
    """
    print(f"Test case: {testCase}")
    
    # Extract features computed by NCPac (mostly related to local structure and geometry)
    print("  Extracting NCPac-computed features...")
    beList, coordList, blList, ba1List, ba2List, btList, CNList, radList, q6q6List, surfCNList, surfList = NCPacFeat(testCase)
    
    # Extract features computed by PySCal (mostly related to bond orientational order)
    print("  Extracting PySCal-computed features...")
    pyscalList, system = pyscalFeat(testCase, CNList)
    
    # Extract features comp uted by SYMMOL (related to point group symmetry)
    print("  Extracting SYMMOL-computed features...")
    symList = symFeat(testCase, system)
    
    # Extract element-specific features
    print("  (Skipped) Extracting element-specific features...")
    # eleList = eleFeat(beList)  # Exclude element-specific features for monometallic nanoparticles for simplicity
    
    # Normalisation of coordinates
    print("  Normalisation of coordinates and radial distance...")
    coordNormList = (coordList-coordList.min()) / (coordList.max()-coordList.min())
    coordNormList.columns = ['xNorm', 'yNorm', 'zNorm']
    radList = (radList-radList.min()) / (radList.max()-radList.min())
    
    print("  Combining all extracted features...")
    dfAll = pd.concat([coordList, coordNormList, beList, 
                       blList, ba1List, ba2List, btList, 
                       CNList, surfCNList, radList, q6q6List, 
                       pyscalList, symList, surfList], axis=1)  # eleList
    
    return dfAll

In [18]:
# beList, coordList, blList, ba1List, ba2List, btList, CNList, radList, q6q6List, surfCNList, surfList = NCPacFeat(testCases[6])
blList['blmin']

0      3.023
1      3.023
2      3.023
3      3.022
4      3.023
       ...  
681    3.024
682    3.023
683    3.023
684    3.023
685    3.023
Name: blmin, Length: 686, dtype: float64

In [ ]:
# Compile total feature set from obtained dataframes and store them
dfAlls = []
for (i, testCase) in enumerate(testCases[len(dfAlls):]): 
    dfAlls.append(allFeat(testCase))

Test case: COS1T000
  Extracting NCPac-computed features...
  Extracting PySCal-computed features...
  Extracting SYMMOL-computed features...
      Progress: 0.0 %
      Progress: 10.0 %
      Progress: 20.0 %
      Progress: 30.0 %
      Progress: 40.0 %
      Progress: 50.0 %
      Progress: 60.0 %
      Progress: 70.0 %
      Progress: 80.0 %
      Progress: 90.0 %
      Progress: 100.0 %
  (Skipped) Extracting element-specific features...
  Normalisation of coordinates and radial distance...
  Combining all extracted features...
Test case: ICS1T000
  Extracting NCPac-computed features...
  Extracting PySCal-computed features...
  Extracting SYMMOL-computed features...
      Progress: 0.0 %
      Progress: 10.0 %
      Progress: 20.0 %
      Progress: 30.0 %
      Progress: 40.0 %
      Progress: 50.0 %
      Progress: 60.0 %
      Progress: 70.0 %
      Progress: 80.0 %
      Progress: 90.0 %
      Progress: 100.0 %
  (Skipped) Extracting element-specific features...
  Normalisatio

<a id='var-store'></a>
## Variables Storage

In [ ]:
# %store -r dfAlls
with open('data/dfAlls.pickle', 'wb') as f: pickle.dump(dfAlls, f)

In [11]:
# Check the output
with open('data/dfAlls.pickle', 'rb') as f: dfAlls = pickle.load(f)
for (i, dfAll) in enumerate(dfAlls):
    print(i, testCases[i])
    display(dfAll.sample(5))

0 COS1T000


,x,y,z,xNorm,yNorm,zNorm,ele,blavg,blmax,blmin,blnum,ba1avg,ba1max,ba1min,ba1num,ba2avg,ba2max,ba2min,ba2num,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,btposnum,btnegnum,cn,gcn,scn,sgcn,rad,q6q6,angParam,entroParam,entroAvgParam,chi1,chi2,chi3,chi4,chi5,chi6,chi7,chi8,chi9,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q8avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord8,disordAvg8,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Iyy,Izz,degenDeg,pg,csm,molRMS,E,Cs,C(inf),Ci,S(inf),C2,surf
603,424.3900,416.2600,416.2600,0.916667,0.583333,0.583333,Pd,2.874,2.874,2.874,12,98.2,180.0,60.0,66,43.0,60.0,1.0,116,91.6,-91.6,0.0,-180.0,180.0,0.0,606,606,12,10.7,0,0.0,0.616329,12,2.999629,-5.337477,-4.711706,6,0,0,0,24,12,0,24,0,0.000020,0.190941,0.574524,0.403915,0.012857,0.600083,0.038470,0.200774,0.574594,0.407370,0.048927,0.601327,2.529689,2.419203,0.379634,0.624598,0.026573,0.028145,0.090697,0.160061,1.800447,2.035720,0.029926,0.048814,1.046005e-07,34.63,33.9,0.74,2,C(inf)v,0.0,0.0,1,inf,2,0,0,0.0,0
487,406.0975,424.3900,418.2925,0.166667,0.916667,0.666667,Pd,2.874,2.874,2.874,9,95.0,180.0,60.0,36,41.4,60.0,1.0,77,95.0,-95.0,0.0,-180.0,180.0,0.0,378,378,9,7.2,6,0.0,0.838150,9,2.999629,-3.799703,-4.064624,2,0,0,0,12,6,0,15,0,0.166667,0.219163,0.595301,0.420240,0.152450,0.611916,0.100708,0.202805,0.582052,0.410302,0.093422,0.604513,2.393774,2.402593,0.642151,0.606680,0.117100,0.080339,0.183920,0.162769,2.332028,2.171236,0.078769,0.063178,1.652345e+01,24.63,24.07,0.57,2,C(inf)v,0.0,0.0,1,inf,2,0,0,0.0,1
69,408.1300,422.3575,406.0975,0.250000,0.833333,0.166667,Pd,2.874,2.874,2.874,9,95.0,180.0,60.0,36,40.1,60.0,1.0,81,97.1,-97.1,0.0,-180.0,180.0,0.0,413,413,9,7.5,6,0.0,0.793353,9,2.249993,-3.799690,-4.261024,2,0,0,0,12,6,0,15,0,0.166667,0.219163,0.595301,0.420240,0.152450,0.611916,0.116667,0.205255,0.584797,0.411995,0.107110,0.605910,2.393734,2.365881,0.642168,0.563421,0.117101,0.089944,0.183912,0.155941,2.332093,2.172865,0.078770,0.064118,1.652345e+01,24.02,23.45,0.58,2,C(inf)v,0.0,0.0,1,inf,2,0,0,0.0,1
93,412.1950,424.3900,404.0650,0.416667,0.916667,0.083333,Pd,2.874,2.874,2.874,9,95.0,180.0,60.0,36,43.2,60.0,1.0,71,92.2,-92.2,0.0,-180.0,180.0,0.0,329,329,9,6.7,6,0.0,0.901734,9,2.916614,-3.799741,-3.773318,3,0,0,0,12,6,0,15,0,0.166667,0.219163,0.595301,0.420240,0.152450,0.611916,0.065465,0.195846,0.577742,0.406565,0.061320,0.601945,2.393774,2.266030,0.642151,0.492203,0.117100,0.065741,0.183920,0.131297,2.332028,2.014834,0.078769,0.051115,1.652213e+01,24.6,24.07,0.54,2,C(inf)v,0.0,0.0,1,inf,2,0,0,0.0,1
665,414.2275,408.1300,422.3575,0.500000,0.250000,0.833333,Pd,2.874,2.874,2.874,12,98.2,180.0,60.0,66,41.0,60.0,1.0,132,93.0,-93.0,0.0,-180.0,180.0,0.0,730,730,12,12.0,0,0.0,0.587428,12,2.916574,-5.337502,-5.337487,6,0,0,0,24,12,0,24,0,0.000020,0.190941,0.574524,0.403915,0.012857,0.600083,0.000014,0.190941,0.574524,0.403915,0.012857,0.600083,1.470609,1.623750,0.379695,0.379685,0.026576,0.026576,0.090668,0.090673,1.802853,1.802482,0.029930,0.029929,2.092010e-07,34.63,33.9,0.74,2,C(inf)v,0.0,0.0,1,inf,2,0,0,0.0,0


1 ICS1T000


,x,y,z,xNorm,yNorm,zNorm,ele,blavg,blmax,blmin,blnum,ba1avg,ba1max,ba1min,ba1num,ba2avg,ba2max,ba2min,ba2num,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,btposnum,btnegnum,cn,gcn,scn,sgcn,rad,q6q6,angParam,entroParam,entroAvgParam,chi1,chi2,chi3,chi4,chi5,chi6,chi7,chi8,chi9,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q8avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord8,disordAvg8,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Iyy,Izz,degenDeg,pg,csm,molRMS,E,C2,Cs,C8,C4,C(inf),surf
809,396.073,391.573,388.792,0.365163,0.210651,0.115163,Pd,2.951,3.0,2.853,9,95.2,180.0,58.0,36,41.9,63.0,1.0,77,89.2,-89.2,0.0,-180.0,180.0,0.0,368,368,9,7.2,6,0.0,0.850467,9,2.713798,-3.643478,-3.900588,3,0,0,0,12,6,0,15,0,0.184263,0.208119,0.590335,0.435013,0.138355,0.601264,0.119601,0.108347,0.487747,0.340241,0.110915,0.552250,2.032247,1.967131,1.867683,1.918627,1.744948,1.705908,1.990581,2.011530,3.097475,2.789803,0.448698,0.370015,1.627819e+01,26.16,25.83,1.43,2,C(inf)v,0.0,0.0,1,0,inf,0,0,2,1
72,402.427,403.000,406.354,0.583333,0.603008,0.718171,Pd,2.927,3.0,2.853,12,97.3,180.0,58.0,66,41.2,63.0,1.0,132,93.3,-93.3,0.0,-180.0,180.0,0.0,757,757,12,12.0,0,0.0,0.433995,8,1.701902,-5.138494,-5.139952,3,0,6,0,21,12,0,24,0,0.021544,0.116453,0.481171,0.316975,0.065255,0.553132,0.013538,0.082139,0.429702,0.255634,0.059898,0.537448,1.902442,1.946067,1.971458,1.997636,1.273074,1.260248,2.014080,2.035244,1.930052,1.866778,0.222375,0.231151,7.734213e+00,35.44,34.51,1.02,2,C(inf)v,0.0,0.0,1,0,inf,0,0,2,0
813,390.292,400.000,389.146,0.166667,0.500000,0.127318,Pd,2.951,3.0,2.853,9,95.2,180.0,58.0,36,41.9,63.0,1.0,77,89.2,-89.2,0.0,-180.0,180.0,0.0,368,368,9,7.2,6,0.0,0.850467,9,2.713639,-3.643443,-3.900566,3,0,0,0,12,6,0,15,0,0.184277,0.208112,0.590328,0.435026,0.138355,0.601249,0.119604,0.108343,0.487743,0.340252,0.110920,0.552241,1.815904,1.846167,2.089155,2.052835,1.806208,1.719275,1.989316,1.984968,3.105924,2.788642,0.459405,0.372380,1.627867e+01,30.82,30.29,1.17,2,C(inf)v,0.0,0.0,1,0,inf,0,0,2,1
275,394.573,393.646,396.073,0.313659,0.281829,0.365163,Pd,2.927,3.0,2.853,12,98.2,180.0,58.0,66,41.3,63.0,1.0,132,90.2,-90.2,0.0,-180.0,180.0,0.0,746,746,12,12.0,0,0.0,0.539136,12,1.701811,-5.138453,-5.138451,6,0,0,0,24,12,0,24,0,0.026392,0.196396,0.569079,0.408448,0.088010,0.586251,0.022514,0.140596,0.498685,0.337754,0.070854,0.560121,1.979800,2.000342,1.928457,1.959809,1.376578,1.320246,2.022837,2.006239,1.945061,1.933446,0.321193,0.274356,2.876514e-07,35.95,35.85,1.7,2,C(inf)v,0.0,0.0,1,0,inf,0,0,2,0
702,401.500,390.073,387.865,0.551504,0.159147,0.083333,Pd,2.963,3.0,2.853,8,90.7,180.0,58.0,28,41.6,63.0,1.0,64,94.1,-94.1,0.0,-180.0,180.0,0.0,292,292,8,6.0,6,0.0,0.919977,4,2.927755,-3.165018,-3.608287,1,0,2,0,8,4,0,13,0,0.186804,0.124295,0.536659,0.376507,0.197412,0.562150,0.085229,0.044087,0.415596,0.262389,0.135013,0.518605,1.977822,2.017250,1.887089,1.981288,2.033700,1.803099,2.164571,2.129853,3.206492,2.831071,0.407960,0.342379,1.501581e+01,24.29,23.97,0.42,2,C8v,4.9119,0.2216,1,1,8.0,4,2,0,1


2 TOS1T000


...

,x,y,z,xNorm,yNorm,zNorm,ele,blavg,blmax,blmin,blnum,ba1avg,ba1max,ba1min,ba1num,ba2avg,ba2max,ba2min,ba2num,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,btposnum,btnegnum,cn,gcn,scn,sgcn,rad,q6q6,angParam,entroParam,entroAvgParam,chi1,chi2,chi3,chi4,chi5,chi6,chi7,chi8,chi9,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q8avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord8,disordAvg8,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Iyy,Izz,degenDeg,pg,csm,molRMS,E,Cs,C8,C4,C2,Ci,S8,S4,C(inf),S(inf),surf
61,403.042,408.825,405.878,0.147068,0.251334,0.235739,Pd,2.711,3.166,2.489,8,90.8,151.0,53.0,28,43.5,79.0,1.0,66,92.0,-92.5,0.0,-180.0,180.0,0.0,285,283,8,6.2,5,0.0,0.821480,2,2.083973,-2.986866,-3.466535,0,0,1,4,4,3,1,8,0,0.242388,0.157894,0.451316,0.336981,0.404714,0.379059,0.101948,0.163422,0.429150,0.236860,0.169989,0.205034,1.490172,2.144059,1.638891,1.556677,0.961613,1.047815,1.829480,1.743004,2.674365,2.075078,1.527192,1.840689,12.984255,11.5,10.88,0.96,2.0,NaN,NaN,NaN,0,0.0,0,0,0.0,0,0,0,0,0,1
354,410.500,412.996,413.097,0.445364,0.411406,0.564908,Pd,2.803,3.189,2.421,12,96.6,169.0,50.0,66,40.8,68.0,2.0,133,92.7,-94.2,0.0,-180.0,180.0,0.0,692,685,12,11.2,0,0.0,0.142961,12,1.174738,-4.372345,-5.102456,4,0,0,2,15,7,1,16,0,0.129673,0.216148,0.510687,0.309961,0.188865,0.388635,0.030431,0.154434,0.456596,0.273300,0.084181,0.274562,2.761447,1.929757,1.005107,0.913010,0.845244,0.727243,1.261992,1.142635,1.853685,1.881083,1.868768,1.672931,4.054254,29.28,27.94,1.97,2,Cs,2.5163,0.1586,1,1.0,0,0,0.0,0,0,0,0,0,0
433,407.004,419.960,415.687,0.305536,0.678666,0.683006,Pd,2.731,2.994,2.558,12,97.4,174.0,50.0,66,41.5,68.0,2.0,135,92.1,-93.5,0.0,-180.0,180.0,0.0,693,683,12,11.3,0,0.0,0.491292,11,1.487817,-5.469536,-5.273211,6,0,0,0,24,11,1,24,0,0.048367,0.188067,0.529891,0.346901,0.175256,0.424231,0.015085,0.129432,0.452797,0.274731,0.073597,0.291125,0.915106,2.174319,0.979183,1.329489,0.462528,0.595765,0.935069,0.977430,2.064091,1.991874,1.258682,1.404241,1.680531,32.13,31.49,2.67,2,C(inf)v,0.0,0.0,1,inf,0,0,0.0,0,0,0,2,0,0
291,400.859,413.404,414.465,0.059755,0.427064,0.627286,Pd,2.768,3.126,2.512,9,90.7,168.0,47.0,36,39.9,77.0,1.0,82,93.9,-97.1,0.0,-180.0,180.0,0.0,344,364,9,7.0,6,0.0,0.757620,8,2.367785,-3.811666,-4.079945,3,0,0,0,11,5,1,16,0,0.158357,0.227843,0.512218,0.335931,0.240561,0.345351,0.112713,0.155789,0.438770,0.260422,0.120502,0.203340,1.819796,1.837875,1.206323,1.191544,0.821088,0.952376,1.771465,1.552984,2.159883,1.993829,1.845320,1.985710,11.974523,15.11,14.11,1.6,2,C(inf)v,0.0,0.0,1,inf,0,0,0.0,0,0,0,2,0,1
138,416.216,417.882,403.326,0.673986,0.598918,0.119374,Pd,2.857,3.207,2.460,13,96.0,173.0,45.0,78,42.7,76.0,3.0,131,91.2,-90.0,0.0,-180.0,180.0,0.0,607,610,13,11.1,0,0.0,0.630624,12,2.585372,-5.239523,-4.436915,2,1,5,0,20,14,0,24,0,0.090875,0.174091,0.379371,0.324394,0.192532,0.324781,0.051795,0.125820,0.434953,0.241484,0.134921,0.243963,2.403761,1.813651,1.371903,1.650248,1.245268,1.273409,1.658522,1.784878,2.032484,2.053507,1.778899,1.961838,9.337689,29.51,28.8,1.3,2,C(inf)v,0.0,0.0,1,inf,0,0,0.0,0,0,0,2,0,0


37 disPd2


,x,y,z,xNorm,yNorm,zNorm,ele,blavg,blmax,blmin,blnum,ba1avg,ba1max,ba1min,ba1num,ba2avg,ba2max,ba2min,ba2num,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,btposnum,btnegnum,cn,gcn,scn,sgcn,rad,q6q6,angParam,entroParam,entroAvgParam,chi1,chi2,chi3,chi4,chi5,chi6,chi7,chi8,chi9,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q8avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord8,disordAvg8,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Iyy,Izz,degenDeg,pg,csm,molRMS,E,Cs,C8,C4,C2,S8,S4,Ci,C(inf),S(inf),surf
271,411.707,413.157,404.620,0.546020,0.576296,0.142684,Pd,2.803,3.225,2.533,12,96.7,173.0,49.0,66,42.7,74.0,1.0,125,90.9,-89.6,0.0,-180.0,180.0,0.0,586,584,12,10.5,0,0.0,0.623445,12,2.603310,-4.655700,-4.557938,3,1,2,0,20,7,1,21,0,0.111493,0.176213,0.493158,0.281155,0.202727,0.385926,0.022286,0.137763,0.466822,0.276829,0.148695,0.361375,1.316198,2.053591,2.514023,2.425583,1.401039,1.519462,2.053939,1.783719,1.303919,1.656010,1.988309,2.016224,6.215785,29.83,29.41,1.79,2,C(inf)v,0.0,0.0,1,inf,0,0,0.0,0,0,0,2,0,0
553,419.698,415.908,407.006,0.807480,0.672334,0.225436,Pd,2.617,2.782,2.409,6,89.7,139.0,62.0,15,43.9,77.0,4.0,49,93.7,-93.0,0.0,-180.0,180.0,0.0,214,219,6,4.2,5,0.0,0.794368,5,2.253828,-2.332431,-3.405098,0,0,0,1,4,2,1,7,0,0.123606,0.286072,0.492715,0.368195,0.442281,0.406168,0.091792,0.152646,0.448689,0.288242,0.153972,0.296754,1.882680,2.070431,2.475368,2.352764,1.475591,1.422735,1.579005,1.646023,2.402736,2.042608,1.863336,1.816262,12.404322,12.26,12.14,0.84,2,C8,13.518,0.3677,1,0.0,4,2,1.0,0,0,0,0,0,1
59,402.135,406.175,408.121,0.232831,0.332554,0.264107,Pd,2.726,3.039,2.449,12,96.9,171.0,54.0,66,44.1,75.0,3.0,115,90.6,-89.1,0.0,-180.0,180.0,0.0,518,513,12,9.8,0,0.0,0.690897,10,1.153825,-5.168526,-4.332545,3,0,3,3,20,13,0,24,0,0.048563,0.077350,0.444747,0.332000,0.111547,0.405548,0.054559,0.125115,0.416809,0.229810,0.109572,0.279364,1.823184,1.760101,1.603890,1.834465,0.910930,1.020325,1.194593,1.267889,1.847780,1.921849,1.587545,1.746282,6.444482,32.79,32.52,1.21,2,C8v,10.0993,0.3178,1,8.0,4,2,1.0,0,0,0,0,0,0
152,401.259,400.980,411.985,0.204168,0.151196,0.398120,Pd,2.742,2.961,2.479,8,88.6,174.0,50.0,28,41.0,78.0,1.0,69,97.2,-96.7,0.0,-180.0,180.0,0.0,308,290,8,5.9,6,0.0,0.830386,8,1.491191,-3.204999,-3.674786,1,0,1,1,7,4,1,13,0,0.156274,0.138373,0.518710,0.395943,0.258402,0.477438,0.094366,0.102943,0.461250,0.280559,0.151735,0.341560,1.017121,1.518147,1.562673,1.823016,0.995680,1.008541,1.212326,1.226858,1.886341,1.849748,1.791007,1.651193,14.031395,17.99,17.69,1.29,2.0,NaN,NaN,NaN,0,0.0,0,0,0.0,0,0,0,0,0,1
93,400.649,421.009,408.677,0.184210,0.850410,0.283391,Pd,2.739,3.135,2.394,7,84.9,141.0,51.0,21,40.1,67.0,3.0,62,95.6,-93.8,0.0,-180.0,180.0,0.0,267,266,7,5.8,5,0.0,0.926654,3,2.914539,-2.593276,-3.676832,0,0,1,0,5,4,1,10,0,0.113752,0.154823,0.460605,0.427318,0.293029,0.404673,0.058984,0.090801,0.410562,0.265818,0.140177,0.228716,1.830294,2.116822,1.795374,1.997266,1.792332,2.003283,1.698710,1.720841,1.926533,2.209745,1.845076,2.045722,16.055814,9.97,9.56,1.16,2,D(inf)h,5.0564,0.2249,1,inf,0,0,inf,0,0,1,2,2,1


38 disPd3


,x,y,z,xNorm,yNorm,zNorm,ele,blavg,blmax,blmin,blnum,ba1avg,ba1max,ba1min,ba1num,ba2avg,ba2max,ba2min,ba2num,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,btposnum,btnegnum,cn,gcn,scn,sgcn,rad,q6q6,angParam,entroParam,entroAvgParam,chi1,chi2,chi3,chi4,chi5,chi6,chi7,chi8,chi9,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q8avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord8,disordAvg8,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Iyy,Izz,degenDeg,pg,csm,molRMS,E,C8,C4,C2,Cs,S4,S8,Ci,C(inf),S(inf),surf
651,402.729,417.812,420.906,0.221751,0.623274,0.609668,Pd,2.807,3.270,2.446,12,96.5,175.0,50.0,66,41.5,75.0,2.0,134,93.2,-91.7,0.0,-180.0,180.0,0.0,690,685,12,11.2,0,0.0,0.557522,9,1.777915,-4.384169,-4.602898,3,1,2,1,20,5,3,20,0,0.108036,0.199020,0.487447,0.296309,0.309370,0.319257,0.042540,0.115039,0.418801,0.226051,0.147467,0.237400,2.577575,1.933668,1.761662,1.780549,1.473469,1.686073,1.851295,1.922539,1.886451,2.054193,1.711518,1.849051,5.845531,29.58,29.51,0.96,2,C(inf)v,0.0,0.0,1,0,0,0.0,inf,0,0,0,2,0,0
180,416.160,405.515,413.297,0.643910,0.232074,0.377743,Pd,2.822,3.288,2.446,12,96.5,174.0,51.0,66,41.4,71.0,2.0,134,91.5,-91.5,0.0,-180.0,180.0,0.0,678,674,12,11.2,0,0.0,0.562242,4,0.751554,-3.953869,-4.527331,1,1,2,3,14,7,0,17,0,0.121917,0.167704,0.471610,0.281836,0.243364,0.358353,0.020549,0.103992,0.361447,0.195228,0.069521,0.179530,2.319758,1.969572,2.007477,1.834136,1.804532,1.727978,1.989278,1.916814,2.176710,1.949065,2.162626,1.968702,7.364777,28.98,28.68,1.29,2,C(inf)v,0.0,0.0,1,0,0,0.0,inf,0,0,0,2,0,0
979,408.902,417.115,428.774,0.415779,0.601101,0.849488,Pd,2.760,2.897,2.573,11,97.1,173.0,53.0,55,43.8,70.0,2.0,115,88.7,-90.5,0.0,-180.0,180.0,0.0,546,536,11,9.7,0,0.0,0.669027,9,2.325455,-4.517398,-4.402359,4,1,0,1,20,7,1,21,0,0.064353,0.203300,0.496856,0.295908,0.221479,0.347385,0.037226,0.118190,0.402396,0.231034,0.070237,0.213847,2.253006,1.839849,2.017131,2.126805,1.841198,1.836792,2.102849,2.027099,2.206391,1.980302,2.211075,2.046523,3.423596,29.97,29.59,1.86,2,C(inf)v,0.0,0.0,1,0,0,0.0,inf,0,0,0,2,0,0
96,397.434,415.022,413.497,0.055320,0.534517,0.383839,Pd,2.817,3.154,2.483,9,91.1,174.0,49.0,36,41.4,67.0,1.0,75,94.8,-88.9,-1.0,-180.0,180.0,1.0,316,321,9,6.5,6,0.0,0.815929,9,1.400368,-3.377285,-3.502087,3,0,0,0,12,5,1,15,0,0.155041,0.227996,0.519737,0.371220,0.193085,0.411998,0.097374,0.159706,0.460084,0.295967,0.119103,0.341285,2.017753,1.960076,1.554212,1.599731,1.627088,1.698456,2.130346,2.006252,1.995946,1.873746,1.686306,1.668755,13.146992,13.53,12.76,1.61,2,C(inf)v,0.0,0.0,1,0,0,0.0,inf,0,0,0,2,0,1
480,404.900,419.504,413.929,0.289989,0.677101,0.397007,Pd,2.829,3.255,2.507,13,96.1,171.0,48.0,78,40.6,77.0,1.0,141,94.3,-92.6,0.0,-180.0,180.0,0.0,695,686,13,11.8,0,0.0,0.499705,9,2.002216,-4.006204,-4.511474,3,0,3,0,14,8,1,16,0,0.093054,0.240266,0.462483,0.274763,0.238485,0.354613,0.038602,0.118177,0.424086,0.222159,0.101856,0.250291,1.523199,2.069685,1.641431,1.679999,1.596918,1.616174,2.030536,2.017196,1.917445,1.998172,1.517136,1.830759,7.853269,26.71,25.44,2.13,2,C8,22.2868,0.4721,1,4,2,1.0,0.0,0,0,0,0,0,0
